## 11.4 Transformer 架构

---
这是第一版没有的内容,好好看看,

---

2017 年开始一个新的架构开始在大多数 nlp 任务中超越 cnn.Transformer (deepl 老是翻译成变形金刚..)

Vaswani 等人的论文 `Attention is all you need` 第一次提到了 Transformer 架构,文章的要点就在标题中: 一个被称为 `neural attention` 的简单机制被证明可用用来构建强大的序列模型,这样模型不具备任何 cnn 或 rnn 的结构.

这一发现引发了 nlp 的一场革命,`neural attention` 的思想迅速成为深度学习最具影响力的思想之一.这一节我们会了解 `neural attention` 是如何工作的,了解它为何对序列数据如此有效.然后我们会创建一个 Transformer 编码器,这是 Transformer 架构的基础组件,随后将其应用在 imdb 电影评论分类任务.


## 理解 self-attention

阅读本书的过程中,你可能会选择略过部分,而认真阅读其他部分,这取决于你的目标或兴趣.如果这个过程能移植到你的模型呢?

模型看到的所有信息对手头任务的重要性并不相同,模型应该多注意某些信息,对其他特征更少关注.

听起来有些熟悉,我们已经在前文中两次接触到了相关的概念

- cnn 中的最大池化,只选择一个最大的特征代替原来整个部分.这是一种全有/全无的形式,只保留最重要特征,抛弃其他.
- tf-idf 归一化,根据不同标记可能携带信息的多少分配重要性标签,重要的标记会有更大的权重,不相关的标记则被淡化.

集中注意力的不同形式可能有很多,但都几乎都是从计算一组特征的重要性开始的.高的数值代表更相关的特征,低的数值代表更不相关的特征,这些数值该如何计算,以及我们该如何使用这些数值,依照不同的方法存在差异.

![general_attention](general_attention.png)

上图显示了深度学习中注意力机制: 输入特征被分配了注意力分数,用于告知输入的下一个表示.


最重要的是这样的关注机制不仅可以用来突出/抹去某些特征,还可以赋予特征上下文感知能力.

上一节中我们了解了词嵌入,词嵌入捕获不同的单词形成具有形状的向量空间,在这个嵌入空间中单词有固定的位置,与空间中其他单词形成固定关系集.但是这并不完全是自然语言的运行法制.自然语言中,一个词具体的含义通常非常依赖上下文的语境.

- 当提到 'date',这个 'date' 究竟是日期,还是约会?
- 提到 'see',究竟是 'I’ll see you soon' 见到,'I’ll see this the project to its end' 见证 'I see what you mean' 了解,直接将上面的 see 全部译为中文 '看见' 再对照译文也是一样的,`see` 和 '看见' 在这些语境下都有些微的差别.
- 又或者完全是指代的词 'it' 'she' 'in' 这些词的意思完全依赖上下文的语境.

显然,一个聪明的嵌入空间将为一个词的不同意思编码为不同的向量,这取决于它周围词的含义.这就是 self-attention 的作用,self-attention 目的是通过序列中相关标记来调节一个标记的表示.这就产生了上下文感知的标记特征.


考虑下面的例子: 'The train left the station on time.' 中的 'station' 究竟是什么含义? 火车站? 广播站? 甚至是国际空间站?这就是一个必须通过上下文语境才能明确含义的词.

![self_attention](self_attention.png)

第一步: 计算 'station' 向量与句子中其他每个词的相关度数字,这些是上文提到的注意力分数.我们简单的使用两个词向量直接的典籍衡量它们之间的关系,这是一个非常有效的计算距离的函数.早在 Transformers 架构出现之前,类似的方式已经是衡量两个词向量距离的标准方式了.当然在实践中,点积的数值还会经过一个缩放函数和 softmax,但是现在在这里,后续的处理都是一些细节了.

第二步: 计算句子中所有单词向量的总和,由相关性分数求加权总和.与 'station' 相关的词对最好结果影响最大(包括 'station' 本身),不相关的词几乎对结果没有什么影响.由此我们得到了一个新向量,这个向量由 'station' 和它的上下文语境而来,特别是它包含了 'train' 这个词向量的一部分,所以向量表示的是 火车站的含义.


如果对句子中每个词都重复这个过程,就产生了一个新的向量序列编码整个句子.下面是这个过程使用 numPy 类似形式的伪代码.

```py
def self_attention(input_sequence):
    output = np.zeros(shape=input_sequence.shape)
    for i, pivot_vector in enumerate(input_sequence): #遍历序列的每一个标记
        scores = np.zeros(shape=(len(input_sequence),))
        for j, vector in enumerate(input_sequence):
            scores[j] = np.dot(pivot_vector, vector.T) #计算这个标记向量与其他标记向量的距离(点积)
        scores /= np.sqrt(input_sequence.shape[1] )#归一化系数
        scores = softmax(scores)
        new_pivot_representation = np.zeros(shape=pivot_vector.shape)
        for j, vector in enumerate(input_sequence):
            new_pivot_representation += vector * scores[j] #加权求和
        output[i] = new_pivot_representation # 最终结果
    return output
```


In [3]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

当然与其他情况相同,keras 早就由内置的实现了 -> MultiHeadAttention 层.示例代码如下:

In [ ]:
# num_heads = 4
# embed_dim = 256
# mha_layer = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
# outputs = mha_layer(inputs, inputs, inputs)

代码有一些奇怪...

- 为什么要反复传入 inputs 3次? 之前没见过这样的用法.
- 为什么这个层叫做 MultiHead?砍掉一个还能长出新的吗?(Hail HYDRA!)

这两个问题都有答案.


### 广义 self-attention: 查询-键-值 模型

到目前为止我们只考虑
